# Optymizery w Deep Learning

## Czym są optymizery?

Optymizery to algorytmy używane do minimalizowania funkcji straty (loss function) poprzez aktualizację wag sieci neuronowej. Ich głównym zadaniem jest znajdowanie optymalnych parametrów modelu w procesie uczenia.

### Podstawowe zasady działania:
- **Gradient Descent**: Podstawowy algorytm optymalizacji, który wykorzystuje gradienty do określenia kierunku aktualizacji wag
- **Learning Rate**: Wielkość kroku, z jakim poruszamy się w kierunku minimum
- **Momentum**: Pomaga w przyspieszeniu konwergencji i unikaniu lokalnych minimów
- **Adaptive Learning**: Automatyczne dostosowywanie learning rate dla różnych parametrów

## Popularne rodzaje optymizatorów

### 1. Stochastic Gradient Descent (SGD)
**Cechy:**
- Najprostszy optymizer
- Stały learning rate
- Może mieć problemy z lokalnimi minimami
- Dobrze sprawdza się z momentum

**Kiedy używać:** Proste problemy, gdy chcemy mieć pełną kontrolę nad procesem uczenia

### 2. Adam (Adaptive Moment Estimation)
**Cechy:**
- Kombinuje momentum i adaptive learning rate
- Automatycznie dostosowuje learning rate dla każdego parametru
- Szybka konwergencja
- Dobrze radzi sobie z rzadkimi gradientami

**Kiedy używać:** Większość problemów deep learning, szczególnie z dużymi sieciami

### 3. RMSprop
**Cechy:**
- Adaptive learning rate
- Dobrze radzi sobie z niestacjonarnymi celami
- Mniej pamięci niż Adam

**Kiedy używać:** RNN, problemy z niestacjonarnymi danymi

### 4. AdaGrad
**Cechy:**
- Dostosowuje learning rate na podstawie historii gradientów
- Learning rate maleje w czasie
- Może przedwcześnie zatrzymać uczenie

**Kiedy używać:** Problemy z rzadkimi cechami, NLP

### 5. AdamW
**Cechy:**
- Poprawiona wersja Adam z prawidłową regularyzacją weight decay
- Lepsza generalizacja
- Popularna w Transformerach

**Kiedy używać:** Duże modele językowe, Transformery

## Przykłady w PyTorch

### Problem regresji - przewidywanie cen domów

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Przygotowanie danych
housing = fetch_california_housing()
X, y = housing.data, housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Konwersja do tensorów PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1)

# Model dla regresji
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_size, 64)
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

# Porównanie różnych optymizatorów
def train_model(optimizer_name, model, optimizer, X_train, y_train, epochs=100):
    criterion = nn.MSELoss()
    losses = []
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            losses.append(loss.item())
            print(f'{optimizer_name} - Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')
    
    return losses

In [2]:
# SGD
model_sgd = RegressionModel(X_train.shape[1])
optimizer_sgd = optim.SGD(model_sgd.parameters(), lr=0.01, momentum=0.9)
print("=== SGD z Momentum ===")
sgd_losses = train_model("SGD", model_sgd, optimizer_sgd, X_train_tensor, y_train_tensor)

# Adam
model_adam = RegressionModel(X_train.shape[1])
optimizer_adam = optim.Adam(model_adam.parameters(), lr=0.001)
print("\n=== Adam ===")
adam_losses = train_model("Adam", model_adam, optimizer_adam, X_train_tensor, y_train_tensor)

# RMSprop
model_rmsprop = RegressionModel(X_train.shape[1])
optimizer_rmsprop = optim.RMSprop(model_rmsprop.parameters(), lr=0.001)
print("\n=== RMSprop ===")
rmsprop_losses = train_model("RMSprop", model_rmsprop, optimizer_rmsprop, X_train_tensor, y_train_tensor)

=== SGD z Momentum ===
SGD - Epoch [0/100], Loss: 5.9667
SGD - Epoch [20/100], Loss: 0.7996
SGD - Epoch [40/100], Loss: 0.6355
SGD - Epoch [60/100], Loss: 0.5832
SGD - Epoch [80/100], Loss: 0.5451

=== Adam ===
Adam - Epoch [0/100], Loss: 4.9593
Adam - Epoch [20/100], Loss: 2.8883
Adam - Epoch [40/100], Loss: 1.3969
Adam - Epoch [60/100], Loss: 0.8766
Adam - Epoch [80/100], Loss: 0.7535

=== RMSprop ===
RMSprop - Epoch [0/100], Loss: 5.7923
RMSprop - Epoch [20/100], Loss: 0.7798
RMSprop - Epoch [40/100], Loss: 0.6209
RMSprop - Epoch [60/100], Loss: 0.5361
RMSprop - Epoch [80/100], Loss: 0.4874


### Problem klasyfikacji - rozpoznawanie cyfr MNIST

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Przygotowanie danych MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Model CNN dla klasyfikacji
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

def train_classifier(model, optimizer, train_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = nn.functional.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % 300 == 0:
                print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.6f}')

def test_classifier(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')
    return accuracy

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:03<00:00, 2.63MB/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 110kB/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:01<00:00, 1.48MB/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 2.21MB/s]


In [4]:
# Porównanie optymizatorów na klasyfikacji

# Adam
model_adam_clf = CNNModel()
optimizer_adam_clf = optim.Adam(model_adam_clf.parameters(), lr=0.001)
print("=== Adam dla klasyfikacji ===")
train_classifier(model_adam_clf, optimizer_adam_clf, train_loader)
adam_acc = test_classifier(model_adam_clf, test_loader)

# AdamW
model_adamw = CNNModel()
optimizer_adamw = optim.AdamW(model_adamw.parameters(), lr=0.001, weight_decay=0.01)
print("\n=== AdamW dla klasyfikacji ===")
train_classifier(model_adamw, optimizer_adamw, train_loader)
adamw_acc = test_classifier(model_adamw, test_loader)

# SGD z schedulером
model_sgd_clf = CNNModel()
optimizer_sgd_clf = optim.SGD(model_sgd_clf.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer_sgd_clf, step_size=3, gamma=0.1)
print("\n=== SGD z schedulером dla klasyfikacji ===")
train_classifier(model_sgd_clf, optimizer_sgd_clf, train_loader)
sgd_acc = test_classifier(model_sgd_clf, test_loader)

/Users/rkorzen/PycharmProjects/pytorch_for_DL_with_python_bootcamp/.venv/lib/python3.12/site-packages/torch/nn/functional.py:1535: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


=== Adam dla klasyfikacji ===
Epoch 1, Batch 0, Loss: 2.313283
Epoch 1, Batch 300, Loss: 0.212340
Epoch 1, Batch 600, Loss: 0.113957
Epoch 1, Batch 900, Loss: 0.077497
Epoch 2, Batch 0, Loss: 0.083085
Epoch 2, Batch 300, Loss: 0.199304
Epoch 2, Batch 600, Loss: 0.095614
Epoch 2, Batch 900, Loss: 0.139638
Epoch 3, Batch 0, Loss: 0.059208
Epoch 3, Batch 300, Loss: 0.064497
Epoch 3, Batch 600, Loss: 0.069703
Epoch 3, Batch 900, Loss: 0.039880
Epoch 4, Batch 0, Loss: 0.063968
Epoch 4, Batch 300, Loss: 0.116923
Epoch 4, Batch 600, Loss: 0.008908
Epoch 4, Batch 900, Loss: 0.138580
Epoch 5, Batch 0, Loss: 0.062773
Epoch 5, Batch 300, Loss: 0.023269
Epoch 5, Batch 600, Loss: 0.026062
Epoch 5, Batch 900, Loss: 0.004383
Test Loss: 0.0309, Accuracy: 99.05%

=== AdamW dla klasyfikacji ===
Epoch 1, Batch 0, Loss: 2.284003
Epoch 1, Batch 300, Loss: 0.155624
Epoch 1, Batch 600, Loss: 0.167865
Epoch 1, Batch 900, Loss: 0.074878
Epoch 2, Batch 0, Loss: 0.054318
Epoch 2, Batch 300, Loss: 0.039031
Epoch 

## Przykłady w TensorFlow/Keras

### Problem regresji

In [5]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Przygotowanie danych (podobnie jak w PyTorch)
housing = fetch_california_housing()
X, y = housing.data, housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_regression_model():
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ])
    return model

def train_and_evaluate(model, optimizer_name, optimizer, epochs=100):
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    print(f"\n=== {optimizer_name} ===")
    history = model.fit(X_train, y_train, 
                       batch_size=32, 
                       epochs=epochs, 
                       validation_split=0.2, 
                       verbose=0)
    
    final_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]
    
    print(f"Final Training Loss: {final_loss:.4f}")
    print(f"Final Validation Loss: {val_loss:.4f}")
    
    return history

In [6]:
# Porównanie różnych optymizatorów w TensorFlow

# SGD
model_sgd_tf = create_regression_model()
optimizer_sgd_tf = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
sgd_history = train_and_evaluate(model_sgd_tf, "SGD", optimizer_sgd_tf)

# Adam
model_adam_tf = create_regression_model()
optimizer_adam_tf = keras.optimizers.Adam(learning_rate=0.001)
adam_history = train_and_evaluate(model_adam_tf, "Adam", optimizer_adam_tf)

# RMSprop
model_rmsprop_tf = create_regression_model()
optimizer_rmsprop_tf = keras.optimizers.RMSprop(learning_rate=0.001)
rmsprop_history = train_and_evaluate(model_rmsprop_tf, "RMSprop", optimizer_rmsprop_tf)

# AdaGrad
model_adagrad_tf = create_regression_model()
optimizer_adagrad_tf = keras.optimizers.Adagrad(learning_rate=0.01)
adagrad_history = train_and_evaluate(model_adagrad_tf, "AdaGrad", optimizer_adagrad_tf)

/Users/rkorzen/PycharmProjects/pytorch_for_DL_with_python_bootcamp/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



=== SGD ===
Final Training Loss: nan
Final Validation Loss: nan

=== Adam ===
Final Training Loss: 0.2256
Final Validation Loss: 0.2844

=== RMSprop ===
Final Training Loss: 0.2339
Final Validation Loss: 0.2834

=== AdaGrad ===
Final Training Loss: 0.2888
Final Validation Loss: 0.3221


### Problem klasyfikacji w TensorFlow

In [7]:
# Ładowanie danych MNIST w TensorFlow
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalizacja danych
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape dla CNN
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

def create_cnn_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

def train_classifier_tf(model, optimizer_name, optimizer, epochs=5):
    model.compile(optimizer=optimizer, 
                 loss='categorical_crossentropy', 
                 metrics=['accuracy'])
    
    print(f"\n=== {optimizer_name} dla klasyfikacji ===")
    history = model.fit(x_train, y_train,
                       batch_size=128,
                       epochs=epochs,
                       validation_data=(x_test, y_test),
                       verbose=1)
    
    return history

In [8]:
# Porównanie optymizatorów na klasyfikacji MNIST

# Adam
model_adam_clf_tf = create_cnn_model()
optimizer_adam_clf_tf = keras.optimizers.Adam(learning_rate=0.001)
adam_history_clf = train_classifier_tf(model_adam_clf_tf, "Adam", optimizer_adam_clf_tf)

# AdamW (dostępny w nowszych wersjach TensorFlow)
try:
    model_adamw_tf = create_cnn_model()
    optimizer_adamw_tf = keras.optimizers.AdamW(learning_rate=0.001, weight_decay=0.01)
    adamw_history_clf = train_classifier_tf(model_adamw_tf, "AdamW", optimizer_adamw_tf)
except AttributeError:
    print("AdamW nie jest dostępny w tej wersji TensorFlow")

# RMSprop
model_rmsprop_clf_tf = create_cnn_model()
optimizer_rmsprop_clf_tf = keras.optimizers.RMSprop(learning_rate=0.001)
rmsprop_history_clf = train_classifier_tf(model_rmsprop_clf_tf, "RMSprop", optimizer_rmsprop_clf_tf)


=== Adam dla klasyfikacji ===
Epoch 1/5


/Users/rkorzen/PycharmProjects/pytorch_for_DL_with_python_bootcamp/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 57ms/step - accuracy: 0.9289 - loss: 0.2344 - val_accuracy: 0.9849 - val_loss: 0.0491
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 49ms/step - accuracy: 0.9756 - loss: 0.0820 - val_accuracy: 0.9879 - val_loss: 0.0370
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 49ms/step - accuracy: 0.9808 - loss: 0.0643 - val_accuracy: 0.9877 - val_loss: 0.0371
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - accuracy: 0.9851 - loss: 0.0490 - val_accuracy: 0.9897 - val_loss: 0.0329
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step - accuracy: 0.9866 - loss: 0.0431 - val_accuracy: 0.9914 - val_loss: 0.0289

=== AdamW dla klasyfikacji ===
Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 52ms/step - accuracy: 0.9194 - loss: 0.2606 - val_accuracy: 0.9813 - val_loss: 0.0566
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - accuracy: 0.9735 - loss: 0.0889 - val_accuracy: 0.9873 - val_loss: 0.0387
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 49ms/step - accuracy: 0.9809 - l

## Porównanie wyników i wnioski

### Kiedy używać którego optymizatora?

1. **Adam** - uniwersalny wybór dla większości problemów
   - Szybka konwergencja
   - Dobra performance na różnych typach danych
   - Automatyczne dostosowywanie learning rate

2. **AdamW** - dla dużych modeli wymagających regularyzacji
   - Lepsza regularyzacja niż Adam
   - Preferowany w Transformerach i dużych modelach

3. **SGD z momentum** - dla prostych problemów i fine-tuningu
   - Dobra generalizacja
   - Wymaga więcej tuningu hiperparametrów
   - Często używany z schedulerami learning rate

4. **RMSprop** - dla RNN i problemów z niestacjonarnymi danymi
   - Dobrze radzi sobie z exploding/vanishing gradients
   - Mniej pamięci niż Adam

### Praktyczne wskazówki:
- Zacznij od **Adam** z lr=0.001
- Jeśli model się przetrenowuje, spróbuj **AdamW**
- Dla prostych problemów testuj **SGD z momentum**
- Zawsze eksperymentuj z różnymi learning rates
- Używaj schedulerów learning rate dla lepszej konwergencji

## Zaawansowane techniki

### Learning Rate Scheduling

In [9]:
# PyTorch - różne typy schedulerów
import torch.optim as optim

model = RegressionModel(X_train.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)

# StepLR - zmniejsza lr co określoną liczbę epok
scheduler_step = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# ExponentialLR - wykładniczo zmniejsza lr
scheduler_exp = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

# ReduceLROnPlateau - zmniejsza lr gdy metryka przestaje się poprawiać
scheduler_plateau = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

# CosineAnnealingLR - lr zmienia się zgodnie z funkcją cosinus
scheduler_cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

print("Różne typy schedulerów są gotowe do użycia!")

Różne typy schedulerów są gotowe do użycia!


In [10]:
# TensorFlow - schedulery learning rate
import tensorflow as tf

# ExponentialDecay
lr_schedule_exp = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

# CosineDecay
lr_schedule_cosine = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.1,
    decay_steps=1000)

# PiecewiseConstantDecay
lr_schedule_piece = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[100, 200],
    values=[0.1, 0.02, 0.005])

# Użycie z optymizatorem
optimizer_scheduled = keras.optimizers.Adam(learning_rate=lr_schedule_exp)

print("Schedulery TensorFlow są gotowe!")

Schedulery TensorFlow są gotowe!


### Gradient Clipping - zapobieganie exploding gradients

In [11]:
# PyTorch - gradient clipping
import torch.nn.utils as utils

model = CNNModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# W pętli treningowej
def train_with_clipping(model, optimizer, data_loader, max_epochs=5):
    model.train()
    for epoch in range(max_epochs):
        for batch_idx, (data, target) in enumerate(data_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = nn.functional.nll_loss(output, target)
            loss.backward()
            
            # Gradient clipping
            utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            if batch_idx % 300 == 0:
                print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.6f}')

print("Funkcja z gradient clipping jest gotowa!")

Funkcja z gradient clipping jest gotowa!


In [12]:
# TensorFlow - gradient clipping
model_tf = create_cnn_model()

# Optimizer z clippingiem
optimizer_clipped = keras.optimizers.Adam(
    learning_rate=0.001,
    clipnorm=1.0  # clipnorm lub clipvalue
)

model_tf.compile(
    optimizer=optimizer_clipped,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model TensorFlow z gradient clipping jest skonfigurowany!")

Model TensorFlow z gradient clipping jest skonfigurowany!


## Podsumowanie

Wybór odpowiedniego optymizatora może znacząco wpłynąć na jakość i szybkość uczenia modelu. **Adam** pozostaje bezpiecznym wyborem dla większości problemów, ale warto eksperymentować z różnymi opcjami w zależności od specyfiki zadania.

### Checklist wyboru optymizatora:
1. ✅ Zacznij od Adam (lr=0.001)
2. ✅ Jeśli model się przetrenowuje → AdamW
3. ✅ Dla RNN → RMSprop
4. ✅ Dla prostych problemów → SGD + momentum
5. ✅ Zawsze używaj schedulerów learning rate
6. ✅ Dodaj gradient clipping dla głębokich sieci
7. ✅ Monitoruj metryki podczas treningu

Pamiętaj: **nie ma jednego najlepszego optymizatora** - wybór zależy od problemu, architektury i danych!